Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [2]:
import pandas as pd

# You may need to change the path
TRAIN_PATH = "/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data/train.csv.zip"
TEST_PATH = "/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data/test.csv"
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [5]:
print(train.shape)
train.head()

(4087, 3)


,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [6]:
print(test.shape)
test.head()

(1022, 2)


,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [8]:
train["description"][0].replace("\n","")

'Sometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [9]:
def wrangle(x):
    """Clean train and test data same way"""
    rows = []
    for row in x["description"]:
        row = row.replace("\n","")
        rows.append(row)
    x["description"] = rows
    return x

In [10]:
train = wrangle(train)
test = wrangle(test)

In [11]:
print(train.shape)
train.head()

(4087, 3)


,id,description,ratingCategory
0,1321,"Sometimes, when whisky is batched, a few lefto...",1
1,3861,An uncommon exclusive bottling of a 6 year old...,0
2,655,This release is a port version of Amrut’s Inte...,1
3,555,This 41 year old single cask was aged in a she...,1
4,1965,"Quite herbal on the nose, with aromas of dried...",1


In [57]:
len(train['description'])

4087

In [71]:
import statistics as stats
rows = []
spaces = []
for row in train['description']:
    size = len(row)
    space = row.count(" ")
    rows.append(size)
    spaces.append(space)
print("Avg num of chars:", stats.mean(rows))
print("Avg num of words:", stats.mean(spaces))

Avg num of chars: 434.393931979447
Avg num of words: 69.8172253486665


In [73]:
n_samples = len(train['description'])
n_words = stats.mean(spaces)
ratio = n_samples / n_words
print(ratio)

58.538562361789154


In [70]:
train['description'][0].count(" ")

75

In [12]:
print(test.shape)
test.head()

(1022, 2)


,id,description
0,3461,Style: Speyside single malt scotch Color: Waln...
1,2604,"Very bright and lively, with a nice balance of..."
2,3341,A new oloroso-forward Chivas positioned to spl...
3,3764,Aged in bourbon casks and then enhanced in Rio...
4,2306,"There is a freshness to the wood on the nose, ..."


In [13]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

### Define Pipeline Components

In [16]:
vect = TfidfVectorizer(stop_words="english", 
                      ngram_range=(1,2), 
                      min_df=5)
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [17]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000,1500,2000),
    'clf__n_estimators': (5,10,15,20),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1280 out of 1280 | elapsed:  2.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=5,
                                                        ngram_range=(1, 2),
                                                        no

In [18]:
grid_search.best_score_

0.7244947673670444

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [19]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [20]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [21]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [22]:
subNumber = 0

In [23]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


In [24]:
grid_search.best_params_

{'clf__max_depth': 15,
 'clf__n_estimators': 20,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

### Define Pipeline Components

In [25]:
from sklearn.decomposition import TruncatedSVD

# vect = TfidfVectorizer we created earlier
# Probably better practice to recreate here

vect = TfidfVectorizer(stop_words="english", 
                       max_df=0.75,
                       max_features=1000,
                       min_df=0.02)

clf = RandomForestClassifier(max_depth=15,
                             n_estimators=20)

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [26]:
lsi = Pipeline([("vect", vect), ("svd", svd)])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [27]:
params = {
    "lsi__svd__n_components": [10,100,250],
    "lsi__vect__ngram_range": [(1,1),(1,2),(1,3)]
}

In [29]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   13.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=0.75,
            

### Make a Submission File

In [30]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [75]:
grid_search.best_params_

{'lsi__svd__n_components': 100, 'lsi__vect__ngram_range': (1, 1)}

In [31]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [32]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [33]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [34]:
# Continue Word Embedding Work Here

import spacy
nlp = spacy.load("en_core_web_lg")

In [36]:
doc = nlp(str(train['description']))

In [37]:
train_vector = doc.vector
print(len(train_vector))

300


In [38]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [39]:
X = get_word_vectors(train['description'])

In [40]:
X_test = get_word_vectors(test['description'])

In [42]:
clf.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
clf.score(X, train['ratingCategory'])

0.9762662099339369

In [44]:
clf.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [50]:
# test = test.drop("ratingCategory", axis=1)

In [46]:
# test['ratingCategory'] = clf.predict(X_test)

### Make a Submission File

In [52]:
# Predictions on test sample
pred = clf.predict(X_test)

In [53]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [54]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [55]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

# Post Lecture Assignment Responses

### 1. What is "Sentiment Analysis"
- Sentiment analysis is an approach to derive emotional / "sentiment" information from text data. This could be useful in many ways, but one example would be in social media. Social media sites typically hsave rules/guidelines for how users can interact with each other, and using sentiment analysis techniques can help them determine if users are being abusive, hateful, or other sentiments that could violate their terms of use.

### 2. Is Document Classification different than "Sentiment Analysis?"
- Document classificiation *is* different than sentiment analysis, but the latter would involve/require the former. Document classification is simply the task of assigning labels/classifying text data, but that does not mean you are also analyzing sentiment. For example, in assignments this week we have used document classification for the purpose of predicting similsr articles, or finding the most common words among a document or group of documents, but none of these explore sentiment. The sentiment piece gets in to the meaning/emotional disposition of words or a document, which as an example could be trying to figure out which commenters on Hacker News are the "saltiest."

### 3. How do we create labeled sentiment data? Are those labels really sentiment?
- One way would be a rules-based approach which could use tokenization and polarity techinques to establish labels. For example, you could determine that documents including profrane words were "negative," and then apply this rule to the entire process. The labels themselves though are not necessarily sentiment, but rather our labeling of words (using polarity - positive vs negative, etc) which could *help* us capture sentiment. A good approach though would likely be to perform frequent sanity checks to ensure labeling is appropriate, and this may even call for review by those with extreme domain knowledge, depending on the subject matter.

### 4. What are common applications of sentiment analysis?
- One common application would be monitoring sentiment of users on social media sites to ensure terms of service are being adhered to. It could also be useful in protecting/improving brand reputation by analyzing sentiment of social media comments, reviews, etc to gauge how people feel about a product/company. Another application would be using sentiment analysis in market research by analyzing survey results at scale to determine best target audiences, most effective marketing campaigns, etc.

### 5. Why did word embeddings work better for the lecture notebook than on the whiskey competition?
- From what I've read it seems to have something to do with the ratio between samples and number of words per sample. I calculated that ratio for the whisky set vs the set used in lecture, and the ratio was much higher for whisky set (around 60 compared to 2 for lecture set). Perhaps the smaller the ratio, the more impactful preprocessing is?

### 6. Neural Networks are becoming more popular for document classification. Why is that the case?
- NNs can work well for document classification due to their non-linearity, so whereas a non-NN classifier may learn patterns including location of words as well as words combos, NN models can learn more about the word combos without caring about location. This could coincide with a greater ability to generalize. For example, a NN model may have performed better on the task of finding similar job listing reviews because length of documents and positions of words would have been less important.

# Try a Different Classifier

In [74]:
from sklearn.naive_bayes import GaussianNB

In [101]:
from sklearn.neural_network import MLPClassifier

In [102]:
vect = TfidfVectorizer(stop_words="english",
                       max_df=0.75,
                       min_df=0.02)

gnb = GaussianNB()

mlp = MLPClassifier(random_state=1)

pipe = Pipeline([('vect', vect), ('mlp', mlp)])

parameters = {
    'vect__max_df': (0.60, 0.70, 0.75, 0.80, 1.0),
    'vect__min_df': (.02, .03, .05),
    'vect__max_features': (500,1000,1500,2000)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.5min finished
/Applications/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.75,
                                                        max_features=None,
                                                        min_df=0.02,
                                                        ngram_range=(1, 1),
                                                      

In [104]:
grid_search.best_score_

0.724492373254168

In [105]:
grid_search.best_params_

{'vect__max_df': 0.7, 'vect__max_features': 500, 'vect__min_df': 0.02}

In [106]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [107]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [108]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [109]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data{subNumber}.csv', index=False)
subNumber += 1

# Logistic Regression

In [110]:
from sklearn.linear_model import LogisticRegressionCV

vect = TfidfVectorizer(stop_words="english",
                       max_df=0.70,
                       min_df=0.02,
                       max_features=500)

gnb = GaussianNB()

lgr = LogisticRegressionCV()

pipe = Pipeline([('vect', vect), ('lgr', lgr)])

parameters = {
    'vect__ngram_range': [(1,1),(1,2),(1,3)]
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   11.1s remaining:    9.7s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   15.3s finished
/Applications/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Applications/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of i

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.7,
                                                        max_features=500,
                                                        min_df=0.02,
                                                        ngram_range=(1, 1),
                                                        

In [111]:
grid_search.best_score_

0.7389288738990822

In [112]:
grid_search.best_params_

{'vect__ngram_range': (1, 1)}

In [113]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [114]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [115]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [116]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module3-document-classification/data{subNumber}.csv', index=False)
subNumber += 1